In [1]:
import pandas as pd
import numpy as np
import datetime
import os
from haversine import haversine
import json
import zipcodes
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))
os.getcwd()


'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_test_balance_Bruce_20190605'

In [2]:
os.listdir(os.getcwd())

['BL_2019Q2 Circ Test zips_7_markets JL_2019-05-24.xlsx',
 'BL 2019Q2 Circ and Existing SOTF Test All Zips 20190605 BR.xlsx',
 'BL_store SOTF existing no TV test 20190531 Test 2 of 3 XT.xlsx',
 '.ipynb_checkpoints',
 'Pull_zips_with_balance.ipynb',
 'BL_zips_stores_of_the_test_JL_2019-06-06.xlsx']

In [3]:
excel_7_mkts=pd.ExcelFile("BL_2019Q2 Circ Test zips_7_markets JL_2019-05-24.xlsx")
print(excel_7_mkts.sheet_names)


df_all_7_store_mkt=pd.DataFrame()
for mkt in excel_7_mkts.sheet_names[2:]:
    df=excel_7_mkts.parse(mkt,dtype=str)
    df_all_7_store_mkt=df_all_7_store_mkt.append(df)
list_stores_of_7_mkt=df_all_7_store_mkt['location_id'].unique().tolist()
print(len(list_stores_of_7_mkt))

# Re-read later because of the store

['unique_per_DMA', 'Circ Test 2019Q2 Zips DeDuped', 'DETROIT', 'GREENSBORO-H.POINT-W.SALEM', 'HARTFORD & NEW HAVEN', 'JACKSONVILLE', 'KANSAS CITY', 'OKLAHOMA CITY', 'PORTLAND, OR']
87


In [4]:
excel_3_mkts=pd.ExcelFile("BL_store SOTF existing no TV test 20190531 Test 2 of 3 XT.xlsx")
print(excel_3_mkts.sheet_names)

df_all_3_stores_mkt=excel_3_mkts.parse("df_1_3_Mkts",dtype=str)

list_test_stores_of_3_mkt=df_all_3_stores_mkt[df_all_3_stores_mkt['group']=="Test"]['location_id'].unique().tolist()
print(len(list_test_stores_of_3_mkt))

['df_1_3_Mkts', 'austin_A', 'austin_A overlap analysis', 'austin_B', 'austin_B overlap analysis', 'phoenix', 'phoenix overlap analysis', 'longisland_A', 'df_1_3_Mkts (2)', 'long island_A overlap analysis', 'longisland_B', 'long island_B overlap analysis', 'longisland_C', 'long island_C overlap analysis', 'df_2_GO_SOTF_0726', 'wide_by_store']
15


In [5]:
df_all_3_stores_mkt.head(2)

,location_id,location_int,Market,ST,SOTF,store_zip,latitude_meas,longitude_meas,revenue_flag,zip,dist_miles,group
0,1154,1154,AUSTIN,TX,Yes,78130,29.6896,-98.1105,P,78130,0.5646947075090354,Control
1,1154,1154,AUSTIN,TX,Yes,78130,29.6896,-98.1105,P,78132,5.995359069896744,Control


In [6]:
all_open_stores=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-06-01.csv",
                            usecols=['location_id','week_end_date','sales'],dtype=str)
all_open_stores=all_open_stores[all_open_stores['location_id']!="6990"]
all_open_stores=all_open_stores[all_open_stores['week_end_date']=="2019-06-01"]
all_open_stores['sales']=all_open_stores['sales'].astype(float)
all_open_stores=all_open_stores[all_open_stores['sales']>0]
print(all_open_stores['week_end_date'].nunique())
all_opening_stores=all_open_stores['location_id'].unique().tolist()


1


In [7]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt",
                         dtype=str,sep="|")
store_list=store_list[['location_id','zip_cd','latitude_meas','longitude_meas']]
store_list['zip_cd']=store_list['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list=store_list[~store_list['location_id'].isin(["6990",'145'])]
store_list=store_list.rename(columns={"zip_cd":"store_zip"})
store_list.shape

(1407, 4)

In [8]:
store_list[~store_list['location_id'].isin(all_opening_stores)]

# Not included, as the sales recently is nan
# 1840 "CLOSED DUE TO HURRICANE - REOPEN 9/19/19" in the Dom's list (planner request folder as Xiao)
# 1918 "TEMP CLOSE SOTF ABATE REOPEN 5/9/19", but no sales observed, in the Dom's list (planner request folder as Xiao)
# 5372 in the tab "2019 SOTF Lists by Wave", same file, wave 2B SOTF G.O. 5/17

,location_id,store_zip,latitude_meas,longitude_meas
632,1840,32446,000000000030.764395,-000000000085.203110
659,1918,95826,000000000038.563259,-000000000121.380165
1306,5372,29485,000000000032.954195,-000000000080.167586


In [9]:
store_list=store_list[store_list['location_id'].isin(all_opening_stores)]
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
store_list.head(2)

,location_id,store_zip,latitude_meas,longitude_meas
0,3,30906,33.421570,-82.018921
1,30,43402,41.363132,-83.650909


In [10]:
Nielsen_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1,usecols=[0,2])
Nielsen_DMA.columns=['store_zip','DMA']
Nielsen_DMA=Nielsen_DMA.drop_duplicates()

In [11]:
store_list_all_DMA=pd.merge(store_list,Nielsen_DMA,on="store_zip",how="left")


list_2_full_DMA=['CHARLOTTE','BUFFALO']
store_list_2_full_DMA=store_list_all_DMA[store_list_all_DMA['DMA'].isin(list_2_full_DMA)]
store_list_2_full_DMA.shape

(37, 5)

In [12]:
mkt_7=['DETROIT', 'GREENSBORO-H.POINT-W.SALEM', 'HARTFORD & NEW HAVEN', 'JACKSONVILLE', 'KANSAS CITY', 'OKLAHOMA CITY', 'PORTLAND, OR']
mkt_3=['AUSTIN','PHOENIX (PRESCOTT)','Long Island']
mkt_2=['BUFFALO','CHARLOTTE']
all_test_mkts=mkt_7+mkt_3+mkt_2

test_stores_multiple_mkts=store_list_all_DMA[store_list_all_DMA['location_id'].isin(store_list_2_full_DMA['location_id'].tolist()+list_stores_of_7_mkt+list_test_stores_of_3_mkt)]

df_stoes_overlapped=test_stores_multiple_mkts.groupby(['location_id'])['DMA'].count().to_frame().reset_index()
df_stoes_overlapped=df_stoes_overlapped[df_stoes_overlapped['DMA']>=2]
df_stoes_overlapped.shape

test_stores_multiple_mkts=test_stores_multiple_mkts[test_stores_multiple_mkts['location_id'].isin(df_stoes_overlapped['location_id'].tolist())]
for loc,group in test_stores_multiple_mkts.groupby(['location_id']):
    print(loc,group['DMA'].tolist(),len([x for x in group['DMA'].tolist() if x in all_test_mkts]))
    
# 5371 only



1977 ['COLUMBIA-JEFFERSON CITY', 'KANSAS CITY'] 1
4556 ['BEND, OR', 'PORTLAND, OR'] 1
5205 ['BUFFALO', 'ELMIRA (CORNING)', 'ROCHESTER, NY'] 1
5226 ['CHARLOTTE', 'COLUMBIA, SC', 'MYRTLE BEACH-FLORENCE'] 1
5371 ['CHARLOTTE', 'GREENSBORO-H.POINT-W.SALEM'] 2
5391 ['CHARLOTTE', 'GREENVLL-SPART-ASHEVLL-AND'] 1


In [13]:
excel_7_mkts=pd.ExcelFile("BL_2019Q2 Circ Test zips_7_markets JL_2019-05-24.xlsx")
print(excel_7_mkts.sheet_names)


df_all_7_store_mkt=pd.DataFrame()
for mkt in excel_7_mkts.sheet_names[2:]:
    df=excel_7_mkts.parse(mkt,dtype=str)
    df=df[df['location_id']!="5371"]
    df_all_7_store_mkt=df_all_7_store_mkt.append(df)
list_stores_of_7_mkt=df_all_7_store_mkt['location_id'].unique().tolist()
print(len(list_stores_of_7_mkt))

['unique_per_DMA', 'Circ Test 2019Q2 Zips DeDuped', 'DETROIT', 'GREENSBORO-H.POINT-W.SALEM', 'HARTFORD & NEW HAVEN', 'JACKSONVILLE', 'KANSAS CITY', 'OKLAHOMA CITY', 'PORTLAND, OR']
86


In [14]:
store_list_all_DMA=pd.merge(store_list,Nielsen_DMA,on="store_zip",how="left")

list_2_full_DMA=['CHARLOTTE','BUFFALO']
store_list_2_full_DMA=store_list_all_DMA[store_list_all_DMA['DMA'].isin(list_2_full_DMA)]
store_list_2_full_DMA=store_list_2_full_DMA[store_list_2_full_DMA['location_id']!="5371"]

In [15]:
store_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                       dtype=str,usecols=['location_id','zip','revenue_flag'])
store_zips=store_zips[store_zips['revenue_flag'].isin(['P','S'])]
store_list_2_full_DMA=pd.merge(store_list_2_full_DMA,store_zips,on="location_id",how="left")

In [16]:
balance_stores=store_list[~store_list['location_id'].isin(list_stores_of_7_mkt)]
balance_stores=balance_stores[~store_list['location_id'].isin(list_test_stores_of_3_mkt)]
balance_stores=balance_stores[~store_list['location_id'].isin(store_list_2_full_DMA['location_id'].tolist())]

df_balance_stores_zips=pd.merge(balance_stores,store_zips,on="location_id",how="left")

cols_list=store_list_2_full_DMA.columns.tolist()+['dist_miles']

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
store_list_2_full_DMA_1=store_list_2_full_DMA[pd.notnull(store_list_2_full_DMA['revenue_flag'])]
store_list_2_full_DMA_2=store_list_2_full_DMA[pd.isnull(store_list_2_full_DMA['revenue_flag'])]


del store_list_2_full_DMA_2['zip']
del store_list_2_full_DMA_2['revenue_flag']



In [18]:
store_list_2_full_DMA_1=store_list_2_full_DMA_1.reset_index()
del store_list_2_full_DMA_1['index']
store_list_2_full_DMA_1['dist_miles']=np.nan

for ind,row in store_list_2_full_DMA_1.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])
    try:
        zip_center=zip_centers[row['zip']]
    except:
        try:
            zip_center=[zipcodes.matching("10044")[0]['lat'],zipcodes.matching("10044")[0]['long']]
        except:
            zip_center=np.nan
    dist=haversine(zip_center,store_center,miles=True)
    store_list_2_full_DMA_1.loc[ind,'dist_miles']=dist
store_list_2_full_DMA_1=store_list_2_full_DMA_1[(store_list_2_full_DMA_1['revenue_flag']=="P") | (store_list_2_full_DMA_1['dist_miles']<=20)]

In [19]:
store_list_2_full_DMA_2=store_list_2_full_DMA_2.reset_index()
del store_list_2_full_DMA_2['index']

df_new_stores_zips=pd.DataFrame()
for ind,row in store_list_2_full_DMA_2.iterrows():
    location_id=row['location_id']
    store_center=[row['latitude_meas'],row['longitude_meas']]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip":zip_cd,"revenue_flag":"zip_in_10","dist_miles":dist},index=[0])
            df_new_stores_zips=df_new_stores_zips.append(df)
            
store_list_2_full_DMA_2=pd.merge(store_list_2_full_DMA_2,df_new_stores_zips,on="location_id",how="left")            


store_list_2_full_DMA=store_list_2_full_DMA_2.append(store_list_2_full_DMA_1)[cols_list]

In [20]:
df_balance_stores_zips_1=df_balance_stores_zips[pd.notnull(df_balance_stores_zips['revenue_flag'])]
df_balance_stores_zips_2=df_balance_stores_zips[pd.isnull(df_balance_stores_zips['revenue_flag'])]


del df_balance_stores_zips_2['zip']
del df_balance_stores_zips_2['revenue_flag']


In [21]:
df_balance_stores_zips_1=df_balance_stores_zips_1.reset_index()
del df_balance_stores_zips_1['index']
df_balance_stores_zips_1['dist_miles']=np.nan

for ind,row in df_balance_stores_zips_1.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])
    try:
        zip_center=zip_centers[row['zip']]
    except:
        try:
            zip_center=[zipcodes.matching("10044")[0]['lat'],zipcodes.matching("10044")[0]['long']]
        except:
            zip_center=np.nan
    dist=haversine(zip_center,store_center,miles=True)
    df_balance_stores_zips_1.loc[ind,'dist_miles']=dist
df_balance_stores_zips_1=df_balance_stores_zips_1[(df_balance_stores_zips_1['revenue_flag']=="P") | (df_balance_stores_zips_1['dist_miles']<=20)]

In [22]:
df_balance_stores_zips_2=df_balance_stores_zips_2.reset_index()
del df_balance_stores_zips_2['index']

df_new_stores_zips=pd.DataFrame()
for ind,row in df_balance_stores_zips_2.iterrows():
    location_id=row['location_id']
    store_center=[row['latitude_meas'],row['longitude_meas']]
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":location_id,"zip":zip_cd,"revenue_flag":"zip_in_10","dist_miles":dist},index=[0])
            df_new_stores_zips=df_new_stores_zips.append(df)
            
df_balance_stores_zips_2=pd.merge(df_balance_stores_zips_2,df_new_stores_zips,on="location_id",how="left")            

balance_cols=cols_list.copy()
cols_list.remove("DMA")
df_balance_stores_zips=df_balance_stores_zips_2.append(df_balance_stores_zips_1)[cols_list]

In [23]:
zip_list_7_mkts_test=df_all_7_store_mkt['zip'].unique().tolist()

zip_list_Austin_test=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="AUSTIN") & (df_all_3_stores_mkt['group']=="Test")]['zip'].unique().tolist()
zip_list_Phoenix_test=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="PHOENIX (PRESCOTT)") & (df_all_3_stores_mkt['group']=="Test")]['zip'].unique().tolist()
zip_list_LongIsland_test=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="Long Island") & (df_all_3_stores_mkt['group']=="Test")]['zip'].unique().tolist()

zip_list_Charlotte_test=store_list_2_full_DMA[store_list_2_full_DMA['DMA']=="CHARLOTTE"]['zip'].unique().tolist()
zip_list_BUFFALO_test=store_list_2_full_DMA[store_list_2_full_DMA['DMA']=="BUFFALO"]['zip'].unique().tolist()

zip_list_balance=df_balance_stores_zips['zip'].unique().tolist()
###
list_of_list_zip=[zip_list_7_mkts_test,zip_list_Austin_test,zip_list_Phoenix_test,zip_list_LongIsland_test,
                          zip_list_Charlotte_test,zip_list_BUFFALO_test,zip_list_balance]
max_rows=max([len(x) for x in list_of_list_zip])

zip_list_7_mkts_test=zip_list_7_mkts_test+[np.nan]*(max_rows-len(zip_list_7_mkts_test))

zip_list_Austin_test=zip_list_Austin_test+[np.nan]*(max_rows-len(zip_list_Austin_test))
zip_list_Phoenix_test=zip_list_Phoenix_test+[np.nan]*(max_rows-len(zip_list_Phoenix_test))
zip_list_LongIsland_test=zip_list_LongIsland_test+[np.nan]*(max_rows-len(zip_list_LongIsland_test))

zip_list_Charlotte_test=zip_list_Charlotte_test+[np.nan]*(max_rows-len(zip_list_Charlotte_test))
zip_list_BUFFALO_test=zip_list_BUFFALO_test+[np.nan]*(max_rows-len(zip_list_BUFFALO_test))

zip_list_balance=zip_list_balance+[np.nan]*(max_rows-len(zip_list_balance))


df_zips=pd.DataFrame({"7 DMAs Full":zip_list_7_mkts_test,"Austin Test only":zip_list_Austin_test,"Phoenix Test only":zip_list_Phoenix_test,
                     "Long Island Test only":zip_list_LongIsland_test,"Charlotte Full DMA":zip_list_Charlotte_test,
                     "Buffalo Full DMA":zip_list_BUFFALO_test,"Balance of the USA":zip_list_balance})


df_zips=df_zips[["7 DMAs Full","Austin Test only","Phoenix Test only","Long Island Test only",
                     "Charlotte Full DMA","Buffalo Full DMA","Balance of the USA"]]

In [24]:
store_list_7_mkts_test=df_all_7_store_mkt['location_id'].unique().tolist()

store_list_Austin_test=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="AUSTIN") & (df_all_3_stores_mkt['group']=="Test")]['location_id'].unique().tolist()
store_list_Phoenix_test=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="PHOENIX (PRESCOTT)") & (df_all_3_stores_mkt['group']=="Test")]['location_id'].unique().tolist()
store_list_LongIsland_test=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="Long Island") & (df_all_3_stores_mkt['group']=="Test")]['location_id'].unique().tolist()

store_list_Charlotte_test=store_list_2_full_DMA[store_list_2_full_DMA['DMA']=="CHARLOTTE"]['location_id'].unique().tolist()
store_list_BUFFALO_test=store_list_2_full_DMA[store_list_2_full_DMA['DMA']=="BUFFALO"]['location_id'].unique().tolist()

store_list_balance=df_balance_stores_zips['location_id'].unique().tolist()

store_list_Austin_control=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="AUSTIN") & (df_all_3_stores_mkt['group']=="Control")]['location_id'].unique().tolist()
store_list_Phoenix_control=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="PHOENIX (PRESCOTT)") & (df_all_3_stores_mkt['group']=="Control")]['location_id'].unique().tolist()
store_list_LongIsland_control=df_all_3_stores_mkt[(df_all_3_stores_mkt['Market']=="Long Island") & (df_all_3_stores_mkt['group']=="Control")]['location_id'].unique().tolist()

###
list_of_list_store=[store_list_7_mkts_test,store_list_Austin_test,store_list_Phoenix_test,store_list_LongIsland_test,
                    store_list_Charlotte_test,store_list_BUFFALO_test,store_list_balance,
                    store_list_Austin_control,store_list_Phoenix_control,store_list_LongIsland_control]
max_rows=max([len(x) for x in list_of_list_store])

store_list_7_mkts_test=store_list_7_mkts_test+[np.nan]*(max_rows-len(store_list_7_mkts_test))

store_list_Austin_test=store_list_Austin_test+[np.nan]*(max_rows-len(store_list_Austin_test))
store_list_Phoenix_test=store_list_Phoenix_test+[np.nan]*(max_rows-len(store_list_Phoenix_test))
store_list_LongIsland_test=store_list_LongIsland_test+[np.nan]*(max_rows-len(store_list_LongIsland_test))

store_list_Charlotte_test=store_list_Charlotte_test+[np.nan]*(max_rows-len(store_list_Charlotte_test))
store_list_BUFFALO_test=store_list_BUFFALO_test+[np.nan]*(max_rows-len(store_list_BUFFALO_test))

store_list_balance=store_list_balance+[np.nan]*(max_rows-len(store_list_balance))

store_list_Austin_control=store_list_Austin_control+[np.nan]*(max_rows-len(store_list_Austin_control))
store_list_Phoenix_control=store_list_Phoenix_control+[np.nan]*(max_rows-len(store_list_Phoenix_control))
store_list_LongIsland_control=store_list_LongIsland_control+[np.nan]*(max_rows-len(store_list_LongIsland_control))


df_stores=pd.DataFrame({"7 DMAs Full":store_list_7_mkts_test,"Austin Test only":store_list_Austin_test,"Phoenix Test only":store_list_Phoenix_test,
                        "Long Island Test only":store_list_LongIsland_test,"Charlotte Full DMA":store_list_Charlotte_test,
                        "Buffalo Full DMA":store_list_BUFFALO_test,"Balance of the USA":store_list_balance,
                        "Austin Control only":store_list_Austin_control,"Phoenix Control only":store_list_Phoenix_control,
                        "Long Island Control only":store_list_LongIsland_control},index=[x for x in range(max_rows)])
df_stores=df_stores[["7 DMAs Full","Austin Test only","Phoenix Test only","Long Island Test only",
                     "Charlotte Full DMA","Buffalo Full DMA","Balance of the USA",
                     "Austin Control only","Phoenix Control only","Long Island Control only"]]

In [25]:
df_stores.columns.tolist()[-3:]

['Austin Control only', 'Phoenix Control only', 'Long Island Control only']

In [26]:
all_values_zip=[]
for col in df_zips.columns.tolist():
    all_values_zip=all_values_zip+df_zips[col].unique().tolist()
all_values_zip = [x for x in all_values_zip if str(x) != 'nan']
all_values_zip=list(set(all_values_zip))


In [27]:
df_all_zips_in=pd.DataFrame({"all_zips":all_values_zip})
df_all_zips_in=df_all_zips_in.reset_index()
del df_all_zips_in['index']

for col in df_zips.columns.tolist():
    df_all_zips_in[col]=np.nan
    df_all_zips_in[col]=df_all_zips_in["all_zips"].apply(lambda x: str(x in df_zips[col].tolist()).replace("True","1").replace("False","0"))
    df_all_zips_in[col]=df_all_zips_in[col].astype(int)
    

In [28]:
all_values_store=[]
for col in df_stores.columns.tolist():
    all_values_store=all_values_store+df_stores[col].unique().tolist()
all_values_store = [x for x in all_values_store if str(x) != 'nan']
all_values_store=list(set(all_values_store))

df_all_stores_in=pd.DataFrame({"all_stores":all_values_store})
df_all_stores_in=df_all_stores_in.reset_index()
del df_all_stores_in['index']

for col in df_stores.columns.tolist():
    df_all_stores_in[col]=np.nan
    df_all_stores_in[col]=df_all_stores_in["all_stores"].apply(lambda x: str(x in df_stores[col].tolist()).replace("True","1").replace("False","0"))
    df_all_stores_in[col]=df_all_stores_in[col].astype(int)
df_all_stores_in['sum']=df_all_stores_in[df_all_stores_in.columns.tolist()[1:]].sum(axis=1)

In [29]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_test_balance_Bruce_20190605/BL_zips_stores_of_the_test_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_zips.to_excel(writer,"zip_codes",index=False)
df_stores.to_excel(writer,"df_stores",index=False)
df_all_zips_in.to_excel(writer,"df_all_zips_compare",index=False)
df_all_stores_in.to_excel(writer,"df_all_stores_compare",index=False)

df_all_7_store_mkt.to_excel(writer,"df_7_mkts",index=False)
df_all_3_stores_mkt.to_excel(writer,"df_3_mkts",index=False)
store_list_2_full_DMA.to_excel(writer,"df_2_DMAs",index=False)
df_balance_stores_zips.to_excel(writer,"df_balance",index=False)

writer.save()
